<a href="https://colab.research.google.com/github/LAMezaCaballero/practica_ETL/blob/main/etl_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from google.colab import files, drive #para cargar archivos desde local a red
from pydrive2.auth import GoogleAuth # usar el googledrive
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# cargar LOS DATOS



---
# segunda version
## tratar de cambiar los datos faltantes:
>  En 'title' hay los valores de 'vineria, fecha, designacion y region'






In [2]:
import os
import pandas as pd


In [3]:
df = pd.read_csv("/content/drive/MyDrive/henry/practicaVinos/winemag-data-130k-v2.csv") # cargar el archivo
df= pd.DataFrame(df) # convertirlo en dataframe
df.drop(columns='Unnamed: 0', axis=0, inplace= True) # eliminar la columna indice extra

In [ ]:
df.drop(columns='region_2', axis=0, inplace= True)# se elimina la columna de reguion_2 por ser valor repetico
df.dropna(axis=0,inplace= True, how= 'all', subset='taster_twitter_handle') # elimina la fila que tenga nan en la columna de taster

In [ ]:
#separar los valores de title por renglon usando extract

df[['winery2', 'year_wine', 'designe', 'region']] = df['title'].str.extract(r'^(.*?) (\d{4}) (.*) \((.*?)\)$')
'''
Elementos clave del regex:

    ^
        Significa "inicio de la cadena". Esto asegura que el patrón comience a buscar desde el principio.

    (.*?)
        Este es el primer grupo de captura.
        .*: Captura cualquier número de caracteres (incluso ninguno).
        ?: Hace que la búsqueda sea "perezosa", es decir, se detiene tan pronto como encuentra un espacio antes del siguiente patrón.
        En este caso, captura el nombre de la bodega (e.g., "Nicosia" o "Quinta dos Avidagos").

    (\d{4})
        Este es el segundo grupo de captura.
        \d{4}: Busca exactamente 4 dígitos (un año como "2013" o "2011").

    (.*)
        Este es el tercer grupo de captura.
        .*: Captura cualquier número de caracteres después del año, hasta llegar al próximo delimitador (en este caso, un paréntesis).
        Captura el nombre del vino (e.g., "Vulkà Bianco" o "Avidagos Red").

    \((.*?)\)
        Este es el cuarto grupo de captura.
        \( y \): Escapan los paréntesis literales en la cadena.
        (.*?): Captura el contenido dentro de los paréntesis (e.g., "Etna" o "Douro").

    $
        Significa "final de la cadena". Esto asegura que el patrón se aplique hasta el final de la línea.'''


'\nElementos clave del regex:\n\n    ^\n        Significa "inicio de la cadena". Esto asegura que el patrón comience a buscar desde el principio.\n\n    (.*?)\n        Este es el primer grupo de captura.\n        .*: Captura cualquier número de caracteres (incluso ninguno).\n        ?: Hace que la búsqueda sea "perezosa", es decir, se detiene tan pronto como encuentra un espacio antes del siguiente patrón.\n        En este caso, captura el nombre de la bodega (e.g., "Nicosia" o "Quinta dos Avidagos").\n\n    (\\d{4})\n        Este es el segundo grupo de captura.\n        \\d{4}: Busca exactamente 4 dígitos (un año como "2013" o "2011").\n\n    (.*)\n        Este es el tercer grupo de captura.\n        .*: Captura cualquier número de caracteres después del año, hasta llegar al próximo delimitador (en este caso, un paréntesis).\n        Captura el nombre del vino (e.g., "Vulkà Bianco" o "Avidagos Red").\n\n    \\((.*?)\\)\n        Este es el cuarto grupo de captura.\n        \\( y \\): E

In [ ]:
df[df['winery2'] != df['winery']].head(2)

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,winery2,year_wine,designe,region
63,France,"This fat, yeasty Champagne is comprised predom...",Brut Rosé,86,58.0,Champagne,Champagne,Roger Voss,@vossroger,Roland Champion NV Brut Rosé (Champagne),Champagne Blend,Roland Champion,NaN,NaN,NaN,NaN
65,France,"From the warm 2015 vintage, this is a soft and...",NaN,86,24.0,Burgundy,Chablis,Roger Voss,@vossroger,Simonnet-Febvre 2015 Chablis,Chardonnay,Simonnet-Febvre,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98758 entries, 0 to 129970
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                98695 non-null  object 
 1   description            98758 non-null  object 
 2   designation            71118 non-null  object 
 3   points                 98758 non-null  int64  
 4   price                  91559 non-null  float64
 5   province               98695 non-null  object 
 6   region_1               79859 non-null  object 
 7   taster_name            98758 non-null  object 
 8   taster_twitter_handle  98758 non-null  object 
 9   title                  98758 non-null  object 
 10  variety                98758 non-null  object 
 11  winery                 98758 non-null  object 
 12  winery2                86302 non-null  object 
 13  year_wine              86302 non-null  object 
 14  designe                86302 non-null  object 
 15  region

---
# Cambiar los valores Vacios

In [ ]:
df["winery2"] = df["winery2"].fillna(df["winery"]) # rellena el winery2 si esta vacio con lo que haya en winery
df[df['winery2'] != df['winery']].head(2)


,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,winery2,year_wine,designe,region
339,Spain,"Red in color, with berry and apple aromas, thi...",1887 Rosado,82,13.0,Catalonia,Cava,Michael Schachner,@wineschach,Cavas Hill NV 1887 Rosado Sparkling (Cava),Sparkling Blend,Cavas Hill,Cavas Hill NV,1887,Rosado Sparkling,Cava
1012,Italy,This ruby-hued blend of Nerello Mascalese and ...,Pietralava,88,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Antichi Vinai 1877 2013 Pietralava Red (Etna),Red Blend,Antichi Vinai 1877,Antichi Vinai,1877,2013 Pietralava Red,Etna


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98758 entries, 0 to 129970
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                98695 non-null  object 
 1   description            98758 non-null  object 
 2   designation            71118 non-null  object 
 3   points                 98758 non-null  int64  
 4   price                  91559 non-null  float64
 5   province               98695 non-null  object 
 6   region_1               79859 non-null  object 
 7   taster_name            98758 non-null  object 
 8   taster_twitter_handle  98758 non-null  object 
 9   title                  98758 non-null  object 
 10  variety                98758 non-null  object 
 11  winery                 98758 non-null  object 
 12  winery2                98758 non-null  object 
 13  year_wine              86302 non-null  object 
 14  designe                86302 non-null  object 
 15  region

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Guardar el DataFrame en una carpeta específica de Drive
ruta_en_drive = '/content/drive/MyDrive/henry/practicaVinos/winemagTransformado.csv'
df.to_csv(ruta_en_drive, index=False)

In [77]:
df = pd.read_csv("/content/drive/MyDrive/henry/practicaVinos/winemagTransformado.csv") # cargar el archivo
df= pd.DataFrame(df) #crea el df
df.head(2)


,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,winery2,year_wine,designe,region
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,Nicosia,2013.0,Vulkà Bianco,Etna
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,Quinta dos Avidagos,2011.0,Avidagos Red,Douro


In [78]:
#eliminar 'winery2' ya que se igualo a 'winery'

df.drop(columns='winery2',axis=1, inplace= True)
df.head(2)

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,year_wine,designe,region
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013.0,Vulkà Bianco,Etna
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011.0,Avidagos Red,Douro


In [79]:
#contar espacios vacios de region1 vs region
print('region_1= ',df['region_1'].isna().sum(),'region= ' ,df['region'].isna().sum())

region_1=  18899 region=  12456


In [80]:
#eso quiere decir que puedo anexar los espacios vacios con los datos de region
df['region_1']=df['region_1'].fillna(df['region'])


In [81]:
#contar espacios vacios de region1 vs region
print('region_1= ',df['region_1'].isna().sum(),'region= ' ,df['region'].isna().sum())
df[df['region_1']!=df['region'] ]

region_1=  909 region=  12456


,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,year_wine,designe,region
36,US,"Aromas of cranberry, barrel spice and herb are...",NaN,86,55.0,Washington,Columbia Valley (WA),Sean P. Sullivan,@wawinereport,Mellisoni 2014 Malbec (Columbia Valley (WA)),Malbec,Mellisoni,2014.0,Malbec (Columbia Valley,WA)
39,US,"The aromas are brooding, with notes of barrel ...",Alder Ridge Vineyard,86,25.0,Washington,Columbia Valley (WA),Sean P. Sullivan,@wawinereport,Ram 2014 Alder Ridge Vineyard Cabernet Franc (...,Cabernet Franc,Ram,2014.0,Alder Ridge Vineyard Cabernet Franc (Columbia ...,WA)
40,France,"This fat, yeasty Champagne is comprised predom...",Brut Rosé,86,58.0,Champagne,Champagne,Roger Voss,@vossroger,Roland Champion NV Brut Rosé (Champagne),Champagne Blend,Roland Champion,NaN,NaN,NaN
42,France,"From the warm 2015 vintage, this is a soft and...",NaN,86,24.0,Burgundy,Chablis,Roger Voss,@vossroger,Simonnet-Febvre 2015 Chablis,Chardonnay,Simonnet-Febvre,NaN,NaN,NaN
43,France,"This soft, rounded wine is ripe with generous ...",NaN,86,15.0,Burgundy,Mâcon-Milly Lamartine,Roger Voss,@vossroger,Vignerons des Terres Secrètes 2015 Mâcon-Mill...,Chardonnay,Vignerons des Terres Secrètes,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98707,US,"The third vintage of Mith is the best yet, com...",Mith Red Wine,91,40.0,Washington,Columbia Valley (WA),Paul Gregutt,@paulgwine,Balboa 2005 Mith Red Wine Red (Columbia Valley...,Red Blend,Balboa,2005.0,Mith Red Wine Red (Columbia Valley,WA)
98711,US,"This ripe, rich, almost decadently thick wine ...",Reserve,91,105.0,Washington,Walla Walla Valley (WA),Paul Gregutt,@paulgwine,Nicholas Cole Cellars 2004 Reserve Red (Walla ...,Red Blend,Nicholas Cole Cellars,2004.0,Reserve Red (Walla Walla Valley,WA)
98717,US,This pure Syrah from Reininger's estate vineya...,Ash Hollow Vineyard,91,41.0,Washington,Walla Walla Valley (WA),Paul Gregutt,@paulgwine,Reininger 2005 Ash Hollow Vineyard Syrah (Wall...,Syrah,Reininger,2005.0,Ash Hollow Vineyard Syrah (Walla Walla Valley,WA)
98718,US,"It's 100% Cabernet Franc, a delightful, bright...",NaN,91,28.0,Washington,Columbia Valley (WA),Paul Gregutt,@paulgwine,Tamarack Cellars 2006 Cabernet Franc (Columbia...,Cabernet Franc,Tamarack Cellars,2006.0,Cabernet Franc (Columbia Valley,WA)


In [82]:
#viendo que las region_1 ya esta mas completa, region es borrable
df.drop(columns='region',axis=1, inplace=True)
df.head(2)

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,year_wine,designe
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013.0,Vulkà Bianco
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,Douro,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011.0,Avidagos Red


In [83]:
#llenaremos los vacios de designation con los valores de designe
df['designation']= df['designation'].fillna(df['designe'])
print('designation= ',df['designation'].isna().sum(),'designe= ' ,df['designe'].isna().sum())#revisar la dif de


designation=  8600 designe=  12456


In [91]:
#borrar la columna designe
df.drop(columns='designe',axis=1, inplace=True)
df.head(2)


,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,year_wine
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013.0
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,Douro,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011.0


In [92]:
#eliminar title
df.drop(columns='title',axis=1, inplace=True)
df.head(2)

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,variety,winery,year_wine
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,White Blend,Nicosia,2013.0
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,Douro,Roger Voss,@vossroger,Portuguese Red,Quinta dos Avidagos,2011.0


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98758 entries, 0 to 98757
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                98695 non-null  object 
 1   description            98758 non-null  object 
 2   designation            90158 non-null  object 
 3   points                 98758 non-null  int64  
 4   price                  91559 non-null  float64
 5   province               98695 non-null  object 
 6   region_1               97849 non-null  object 
 7   taster_name            98758 non-null  object 
 8   taster_twitter_handle  98758 non-null  object 
 9   variety                98758 non-null  object 
 10  winery                 98758 non-null  object 
 11  year_wine              86302 non-null  float64
dtypes: float64(2), int64(1), object(9)
memory usage: 9.0+ MB


In [98]:
# Guardar el DataFrame en una carpeta específica de Drive
ruta_en_drive = '/content/drive/MyDrive/henry/practicaVinos/winemagTransformado.csv'
df.to_csv(ruta_en_drive, index=False)

In [97]:
#eliminar filas con elementos vacios
df.dropna(axis=0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81039 entries, 1 to 98757
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                81039 non-null  object 
 1   description            81039 non-null  object 
 2   designation            81039 non-null  object 
 3   points                 81039 non-null  int64  
 4   price                  81039 non-null  float64
 5   province               81039 non-null  object 
 6   region_1               81039 non-null  object 
 7   taster_name            81039 non-null  object 
 8   taster_twitter_handle  81039 non-null  object 
 9   variety                81039 non-null  object 
 10  winery                 81039 non-null  object 
 11  year_wine              81039 non-null  float64
dtypes: float64(2), int64(1), object(9)
memory usage: 8.0+ MB


In [ ]:
!cd /content/tu-repositorio && git add .
!cd /content/tu-repositorio && git commit -m "Actualización desde Colab"
!cd /content/tu-repositorio && git push